## Create the metadata to alias

In this example, we create two Operators that we will subsequently alias.

In [4]:
from ni.datastore.client import Client
from ni.measurements.metadata.v1.metadata_store_pb2 import Operator

client = Client()
operatorOne = Operator(operator_name="Jane Doe")
operatorTwo = Operator(operator_name="John Smith")

client.create_operator(operatorOne)
client.create_operator(operatorTwo)

'73326cec-db0b-4f84-9d48-24005fa04895'

## Register aliases for the desired metadata

Note: The metadata being aliased must have already been created prior to registering the alias.

In [5]:
client.create_alias("primary_operator", operatorOne)
client.create_alias("secondary_operator", operatorTwo)

print("Aliases registered successfully.")

Aliases registered successfully.


## Reference the alias

You can reference the alias in place of the metadata ID when creating further metadata or publishing.

In [9]:
from ni.measurements.data.v1.data_store_pb2 import TestResult

test_result = TestResult(
    test_result_name="Test Result with Operator Alias",
    operator_id="secondary_operator"
)

test_result_id = client.create_test_result(test_result)

print("The test results was created successfully using an alias to reference an operator.")


The test results was created successfully using an alias to reference an operator.


## Verify expected alias usage

In [10]:
# Retrieve the session and verify which operator was associated with it
retrieved_session = client.get_test_result(test_result_id)
retrieved_operator_id = retrieved_session.operator_id
operator_details = client.get_operator(retrieved_operator_id)

print(f"The name of the operator for the session: {operator_details.operator_name}")

The name of the operator for the session: John Smith


## Resolve alias

Look up the target information of the alias (i.e. what the alias is an alias for)

In [11]:
alias = client.get_alias("primary_operator")

print(alias)

alias_name: "primary_operator"
target_type: ALIAS_TARGET_TYPE_OPERATOR
target_id: "2d7e559f-0614-47db-aa59-622d9c0d5af4"



## Query all aliases

In [12]:
aliases = client.query_aliases("")
for alias in aliases:
    print(alias)

alias_name: "primary_operator"
target_type: ALIAS_TARGET_TYPE_OPERATOR
target_id: "2d7e559f-0614-47db-aa59-622d9c0d5af4"

alias_name: "secondary_operator"
target_type: ALIAS_TARGET_TYPE_OPERATOR
target_id: "73326cec-db0b-4f84-9d48-24005fa04895"



## Query aliases with filter

In [13]:
aliases = client.query_aliases("$filter=name eq 'primary_operator'")
for alias in aliases:
    print(alias)

alias_name: "primary_operator"
target_type: ALIAS_TARGET_TYPE_OPERATOR
target_id: "2d7e559f-0614-47db-aa59-622d9c0d5af4"



## Unregister alias

Unregisters an existing alias on the system. This does not affect the underlying target/metadata to which the alias refers.

In [14]:
unregistered = client.delete_alias("primary_operator")

print(f"Success of unregistering 'primary_operator': {unregistered}\n")

## Query the aliases again to confirm the alias has been removed
aliases = client.query_aliases("")
for alias in aliases:
    print(alias)

# Try to unregister the alias again. This should not succeed since the alias has already been removed.
unregistered = client.delete_alias("primary_operator")
print(f"Success of unregistering already-removed 'primary_operator': {unregistered}")

Success of unregistering 'primary_operator': True

alias_name: "secondary_operator"
target_type: ALIAS_TARGET_TYPE_OPERATOR
target_id: "73326cec-db0b-4f84-9d48-24005fa04895"

Success of unregistering already-removed 'primary_operator': False
